In [1]:
run ~/Projects/replab/replab_addpaths(2,1); % replace by your own path
install_sdpt3;

Adding VPI to the path
Adding MOxUnit to the path
Adding embedded YALMIP to the path
Adding embedded SDPT3 to the path. Please run 'install_sdpt3' to complete the setup of this solver
Adding MOcov to the path

---------------------------------------------------------------------------
SDPT3 installation script
   Directory: /home/denis/Projects/replab/external/SDPT3
   Matlab 9.4.0.813654 (R2018a) on GLNXA64
---------------------------------------------------------------------------
Looking for existing binaries...found!
   If for some reason you need to rebuild the binaries, use this command:
      install_sdpt3 -rebuild
---------------------------------------------------------------------------
Adding SDPT3 to the Matlab path:
   Base...already there.
   Solver...added.
   HSDSolver...added.
   Binaries...added.
   Examples...added.
Please save the Matlab path if you want to use SDPT3 from any directory.
---------------------------------------------------------------------------
SDPT

# Implementing the cut inflation test

We are using the enumeration of variables

$\vec{P}^\text{test}_\text{ABC} = \big( P(abc) \big) = \big(P(000), P(001), P(010), P(011), P(100), P(101), P(110), P(111)\big)$,

for the triangle with connections (shown here flattened, there is a single `beta` variable):

```
beta -> A <- gamma -> B <- alpha -> C <- beta (looping)
```

For example, we test the distributions:

$ \vec{P}^\text{equal} = \big( 1/2, 0, 0, 0, 0, 0, 0, 1/2 \big). $

$ \vec{P}^\text{notequal} = \big( 0, 1/6, 1/6, 1/6, 1/6, 1/6, 1/6, 0 \big). $

and a new distribution inspired by the $W$ state:

$ \vec{P}^\text{W} = \big( 0, 1/3, 1/3, 0, 1/3, 0, 0, 0 \big). $

We will test those distributions using the cut inflation 

$\vec{P}^\text{inf}_\text{A1B1C1} = \big( P(a_1 b_1 c_1) \big) = \big(P(000), P(001), P(010), P(011), P(100), P(101), P(110), P(111)\big)$,

which is connected in this way:

```
beta1 -> A1 <- gamma1 -> B1 <- alpha1 -> C1 <- beta2
```.

We now defined the distribution being tested, the inflation distribution $\vec{P}^\text{test}$ with its first constraint, namely nonnegativity.

In [2]:
P_abc = [1 0 0 0 0 0 0 1]'/2; % we omit the "test" superscript, if the distribution is on a,b,c it is the test one
P_a1b1c1 = sdpvar(8, 1); % we omit the "inf" superscript, if the distribution is on a1,b1,c1 it is the inflation one
Constraint_Pos = [P_a1b1c1 >= 0];

Now the hard part. We need to add the constraints that match marginal distributions of $\vec{P}^\text{test}$ with parts of $\vec{P}^\text{inf}$. For example, we match $\vec{P}^\text{inf}_\text{A1C1}$

In [3]:
P_a = [1 1 1 1 0 0 0 0
       0 0 0 0 1 1 1 1] * P_abc;
P_c = [1 0 1 0 1 0 1 0
       0 1 0 1 0 1 0 1] * P_abc;
P_a1c1 = [1 0 1 0 0 0 0 0
          0 1 0 1 0 0 0 0
          0 0 0 0 1 0 1 0
          0 0 0 0 0 1 0 1] * P_a1b1c1;
Constraint_A1C1 = [P_a1c1 == [P_a(1)*P_c(1); P_a(1)*P_c(2); P_a(2)*P_c(1); P_a(2)*P_c(2)]];

Now, write the constraints for $\vec{P}^\text{test}_\text{AB} = \vec{P}^\text{inf}_\text{A1B1}$ and $\vec{P}^\text{test}_\text{BC} = \vec{P}^\text{inf}_\text{B1C1}$.

In [4]:
P_ab = [];
P_a1b1 = [];
Constraint_A1B1 = [P_ab == P_a1b1];
P_bc = [];
P_b1c1 = [];
Constraint_B1C1 = [P_bc == P_b1c1];

We now solve the feasibility problem. Note that we fix the objective to `0`.

In [5]:
CONS = [Constraint_Pos
        Constraint_A1C1
        Constraint_A1B1
        Constraint_B1C1];
optimize(CONS, 0)



Optimal solution found.


ans = 

  struct with fields:

    yalmipversion: '20190425'
       yalmiptime: 1.7091
       solvertime: 0.4351
             info: 'Successfully solved (LINPROG)'
          problem: 0



## And now?
Try to check compatibility of the `notequal` and `W` distributions.
You should find that `notequal` is compatible: we know there exists an explicit model!
What happens for `W`? In the lab session, we'll prove using another inflation that it is not compatible with the triangle causal structure. But is it compatible with the cut inflation?